<a href="https://colab.research.google.com/github/GabrielBonilla/SantanderDev/blob/main/SantanderDevWeek2023_Gabriel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Santander Dev Week 2023 (ETL com Python)

**Contexto:** Você é um cientista de dados no Santander e recebeu a tarefa de envolver seus clientes de maneira mais personalizada. Seu objetivo é usar o poder da IA Generativa para criar mensagens de marketing personalizadas que serão entregues a cada cliente.

**Condições do Problema:**

1. Você recebeu uma planilha simples, em formato CSV ('SDW2023.csv'), com uma lista de IDs de usuário do banco:
  ```
  UserID
  1
  2
  3
  4
  5
  ```
2. Seu trabalho é consumir o endpoint `GET https://sdw-2023-prd.up.railway.app/users/{id}` (API da Santander Dev Week 2023) para obter os dados de cada cliente.
3. Depois de obter os dados dos clientes, você vai usar a API do ChatGPT (OpenAI) para gerar uma mensagem de marketing personalizada para cada cliente. Essa mensagem deve enfatizar a importância dos investimentos.
4. Uma vez que a mensagem para cada cliente esteja pronta, você vai enviar essas informações de volta para a API, atualizando a lista de "news" de cada usuário usando o endpoint `PUT https://sdw-2023-prd.up.railway.app/users/{id}`.



In [ ]:
# Utilize sua própria URL se quiser ;)
# Repositório da API: https://github.com/digitalinnovationone/santander-dev-week-2023-api
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

## **E**xtract

Extraia a lista de IDs de usuário a partir do arquivo CSV. Para cada ID, faça uma requisição GET para obter os dados do usuário correspondente.

In [ ]:
import pandas as pd

df = pd.read_csv('SDW2023.csv')
user_ids = df['UserID'].tolist()
print(user_ids)

[1168, 1169, 1170, 1422]


In [ ]:
import requests
import json

def get_user(id):
  response = requests.get(f'{sdw2023_api_url}/users/{id}')
  return response.json() if response.status_code == 200 else None

users = [user for id in user_ids if (user := get_user(id)) is not None]
print(json.dumps(users, indent=2))

[
  {
    "id": 1168,
    "name": "Cornelios",
    "account": {
      "id": 1243,
      "number": "0099",
      "agency": "0001",
      "balance": 500.0,
      "limit": 1000.0
    },
    "card": {
      "id": 1142,
      "number": "**** **** **** 9076",
      "limit": 1500.0
    },
    "features": [],
    "news": [
      {
        "id": 2966,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
        "description": "Seu limite pode lhe garantir milhas para viagens, entre em contato com nossos atendentes para saber mais"
      },
      {
        "id": 2967,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
        "description": "Ol\u00e1 Cornelios Seu limite pode lhe garantir milhas para viagens, entre em contato com nossos atendentes para saber mais"
      },
      {
        "id": 2968,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit

## **T**ransform

Utilize a API do OpenAI GPT-4 para gerar uma mensagem de marketing personalizada para cada usuário.

In [ ]:
def generate_news(user):
  limite = user['card']
  if limite['limit']  >= 1000:
    mensagem = f"Olá {user['name']} seu limite pode lhe garantir milhas para viagens, entre em contato com nossos atendentes para saber mais"
  else:
    mensagem = f"Olá {user['name']} você sabia que utilizar seu cartâo de crédito para compras pode aumentar limite de crédito?"
  return mensagem
for user in users:
  news = generate_news(user)
  print(news)
  user['news'].append({
      "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
      "description": news
  })

Olá Cornelios seu limite pode lhe garantir milhas para viagens, entre em contato com nossos atendentes para saber mais
Olá Mano Menezes seu limite pode lhe garantir milhas para viagens, entre em contato com nossos atendentes para saber mais
Olá Enner Violencia seu limite pode lhe garantir milhas para viagens, entre em contato com nossos atendentes para saber mais
Olá Josneiverson você sabia que utilizar seu cartâo de crédito para compras pode aumentar limite de crédito?


## **L**oad

Atualize a lista de "news" de cada usuário na API com a nova mensagem gerada.

In [ ]:
def update_user(user):
  response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
  return True if response.status_code == 200 else False

for user in users:
  success = update_user(user)
  print(f"User {user['name']} updated? {success}!")

User Cornelios updated? True!
User Mano Menezes updated? True!
User Enner Violencia updated? True!
User Josneiverson updated? True!
